In [2]:
# 라이브러리 import

import pandas as pd
from tqdm import tqdm

In [3]:
# 파일 불러오기

df = pd.read_csv('아이돌인스타그램_clustering.csv', encoding='utf-8-sig')
del df['Unnamed: 0']
df.head()

,index,time,doc,url,token,text_noun,cluster
0,0,2022. 5. 9. 13:10,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,https://blog.naver.com/syr03051/222723113775,"['드라마', '설강화', '배우', '김혜윤', '배우', '김혜윤']","['드라마', '설강화', '배우', '김혜윤', '배우', '김혜윤']",2
1,1,2022. 9. 29. 8:10,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,https://blog.naver.com/syr03051/222885506031,"['인스타', '트위터', '댓글', '프루덴셜', '센터']","['인스타', '트위터', '댓글', '프루덴셜', '센터']",1
2,2,2022. 9. 2. 14:52,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,https://blog.naver.com/fenergy12/222864930875,"['사진', '모음', '사진', '모음', '사진', '모음', '사진', '모음']","['사진', '모음', '사진', '모음', '사진', '모음', '사진', '모음']",2
3,3,2022. 5. 7. 1:03,게시판 로제 인스타그램을 박채영 로제,https://blog.naver.com/199651/222723356800,['게시판'],['게시판'],2
4,4,2022. 1. 15. 16:42,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,https://blog.naver.com/alwayshappiness211/2226...,"['스토리', '포에버', '팔로워', '팔로잉', '게시', '물']","['스토리', '포에버', '팔로워', '팔로잉', '게시', '물']",3


In [4]:
df.rename(columns={'doc':'sents'}, inplace=True)
#df.rename(columns={'클러스터':'cluster'}, inplace=True)
df=df[['sents','cluster']]
df

,sents,cluster
0,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,2
1,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,1
2,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,2
3,게시판 로제 인스타그램을 박채영 로제,2
4,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,3
...,...,...
7084,크리스탈 폴로 셔츠 포니 가방 캐주얼룩 완성 걸스픽 크리스탈 폴로 ...,0
7085,스타일링 따라하고 싶은 셀럽들의 폴로 랄프로렌 셔츠 스타일링 크리스탈 기은세 꾸리...,0
7086,게시판 크리스탈 정수정 근황 좋아보여요 광고실행사 배우 정수정 및 크리스탈...,0
7087,스타패션 크리스탈 정수정 인스타그램 스트라이프 럭비 티셔츠 폴로 랄프로렌 ...,0


In [5]:
#파일 저장
import pickle
df.to_pickle('아이돌인스타그램_lda전.pkl')
df.to_csv('아이돌인스타그램_lda전.csv', encoding='utf-8-sig')

In [6]:
'''형태소 분석'''

from konlpy.tag import Okt
okt = Okt()

#명동형부 main_pos
def postagging(dataframe):
    main_pos = [] #데이터프레임의 새 컬럼이 될 리스트 생성
    for sentence in tqdm(df['sents']): #리뷰들을 하나씩 가지고 오기
        pos = okt.pos(sentence, norm=True, stem=True) #형태소 분석
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in 
                      ('Adjective', #형용사
                      'Noun', #명사
                      'Adverb', #부사
                      'Verb', #동사
                      )]
        
        main_word_str = " ".join(main_words) 
        #main_words리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
        main_pos.append(main_word_str)
        #선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가
        df['main_pos'] = pd.DataFrame(main_pos)
        #리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가

In [7]:
postagging(df)

100%|██████████| 7089/7089 [02:33<00:00, 46.18it/s] 


In [8]:
### 명사 noun

def postagging2(dataframe):
    noun = [] # 명사 리스트
    for sentence in tqdm(dataframe['main_pos']): 
        pos = okt.pos(sentence) # 형태소 분석
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ('Noun')] # 명사
        
        main_words_str = " ".join(main_words) 
        noun.append(main_words_str) 
        dataframe["noun"] = pd.DataFrame(noun) 

postagging2(df)

100%|██████████| 7089/7089 [00:51<00:00, 138.53it/s]


In [9]:
df = df[['sents','main_pos','noun','cluster']]
df.head()

,sents,main_pos,noun,cluster
0,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,지수 드라마 설강화 배우 김혜윤 인스타그램 지수 배우 김혜윤 인스타그램 지수,지수 드라마 설강화 배우 김혜윤 인스타그램 지수 배우 김혜윤 인스타그램 지수,2
1,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 리사 인스타그램 리사...,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 리사 인스타그램 리사...,1
2,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,지수 인스타그램 사진 모음 지수 인스타그램 사진 모음 지수 인스타그램 사진 모음 지...,지수 인스타그램 사진 모음 지수 인스타그램 사진 모음 지수 인스타그램 사진 모음 지...,2
3,게시판 로제 인스타그램을 박채영 로제,게시판 로제 인스타그램 박채영 로제,게시판 로제 인스타그램 박채영 로제,2
4,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 명 팔로잉 명 게시 물 개 님,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 명 팔로잉 명 게시 물 개,3


In [10]:
#한국어불용어리스트
#출처:https://mr-doosun.tistory.com/24
#출처: https://wikidocs.net/77135

stopwords = '있어요 입니다 해서 그냥 같아요 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하'.split() 

# TF-IDF

In [11]:
# 클러스터 별로 하나의 document를 만들어줌

# 명동형부 tf-idf

# doc0 = ' '.join(df[df['cluster']==0].main_pos)
# doc1 = ' '.join(df[df['cluster']==1].main_pos)
# doc2 = ' '.join(df[df['cluster']==2].main_pos)

# 명사로만 tf-idf
doc0 = ' '.join(df[df['cluster']==0].noun)
doc1 = ' '.join(df[df['cluster']==1].noun)
doc2 = ' '.join(df[df['cluster']==2].noun)

In [12]:
docs = [doc0,doc1,doc2]

In [13]:
# TfidfVectorizer를 활용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=500, stop_words=stopwords)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)

In [14]:
# 파일 저장

nums = range(0,3) ## cluster 개수 지정

def tfidfv(documents):
    for num in nums:
        path = 'tf_idf_cluster_명_%d.csv'%num #위치

        #tfidfvectorizer
        first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[num] 
        tfidf_df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), 
                                index=tfidf_vectorizer.get_feature_names(), 
                                columns=["tfidf"])
        a = tfidf_df.sort_values(by=["tfidf"],ascending=False)
        a.to_csv(path,encoding='utf-8-sig')
        
tfidfv(docs)

c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
# 명동형부 tf-idf

doc0 = ' '.join(df[df['cluster']==0].main_pos)
doc1 = ' '.join(df[df['cluster']==1].main_pos)
doc2 = ' '.join(df[df['cluster']==2].main_pos)

In [16]:
docs = [doc0,doc1,doc2]

In [17]:
# TfidfVectorizer를 활용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=500, stop_words=stopwords)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)

In [18]:
# 파일 저장

nums = range(0,3) ## cluster 개수 지정

def tfidfv(documents):
    for num in nums:
        path = 'tf_idf_cluster_명동형부_%d.csv'%num #위치

        #tfidfvectorizer
        first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[num] 
        tfidf_df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), 
                                index=tfidf_vectorizer.get_feature_names(), 
                                columns=["tfidf"])
        a = tfidf_df.sort_values(by=["tfidf"],ascending=False)
        a.to_csv(path,encoding='utf-8-sig')
        
tfidfv(docs)

c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
